stage 3

this notebook has three functions - 
1. get_fire_dates(fires) - get a list of dates from the FiresWithEuc dataframe
2. get_bbox(fire) - get the bounding boxes from the same dataframe - these are both complete and functioning 
3. def get_sat_images(bounds, dates, fires) - the previous two functions to query satelitte imagery 
 

In [1]:
#resources used
#https://stackoverflow.com/questions/6871016/adding-days-to-a-date-in-python

In [2]:
#!pip install --upgrade stackstac==0.5.0

In [3]:
import datetime
import geopandas as gpd # 0.13.2
import stackstac # Needs to be 0.5.0 - if it is not run "conda install -c conda-forge stackstac" in MPC terminal, also the pip install in above codeblock
import dask.array as da
from pystac_client import Client
import leafmap
import pystac_client
import dask
import planetary_computer as pc
import geogif
import numpy as np
import xarray as xr
import rioxarray
import cartopy.crs as ccrs
import pyproj

In [4]:
#stackstac.__version__

In [5]:
#import shapfiles
#EucList= gpd.read_file("Euc/EucInFires.shp")
FiresWithEuc = gpd.read_file("Fire/FiresWithEuc.shp")

In [6]:
#import matplotlib.pyplot as plt
#FiresWithEuc.explore("ALARM_DATE", legend=False)

In [7]:
# bounds for each bounding box
#FiresWithEuc.head(10).bounds

In [8]:
#FiresWithEuc.crs

--------------------------------------------------

In [18]:
def get_fire_dates(fires):
    '''
    This function creates a list of alarm date for fires and calculates and outputs the date range for use in sat imagery queries
        start date - alarm date minus 30 days
        end date - alarm date plus 30 days
        
    args:
    fires: is a geodataframe that contains the column 'ALARM_DATE' (should be in "%Y-%m-%d") for each fire
        if you are using differnt data then you may need to change the column name 

    this will then be used for queries landsat satelitte images in a later function
    '''
    
    #create a list with the alarm_dates
    alarm_dates = fires['ALARM_DATE'] # you change the name of this column if using other data
    alarm_dates_list = list(alarm_dates)
    
    #lists to store start and end dates
    start_dates = []  
    end_dates = []    
    
    # Calculate start and end dates within 30 day month range
    for alarm_date in alarm_dates_list:
        datex = datetime.datetime.strptime(alarm_date, "%Y-%m-%d")
        
        # calculates and formats start dates
        start_date = datex - datetime.timedelta(days=30)
        start_date = start_date.strftime("%Y-%m-%d")
        
        # calculates and formats end dates
        end_date = datex + datetime.timedelta(days=30)
        end_date = end_date.strftime("%Y-%m-%d")
        
        #print(f"Alarm Date: {alarm_date}, Start Date: {start_date}, End Date: {end_date}")
        
        #appends list
        start_dates.append(start_date) 
        end_dates.append(end_date)
        
        
    
    return start_dates, end_dates   


get_fire_dates(FiresWithEuc)

(['2020-07-17',
  '2020-07-17',
  '2020-01-29',
  '2005-06-19',
  '2005-06-26',
  '2008-05-21',
  '2008-05-12',
  '2008-05-23',
  '2009-07-13',
  '2010-05-19',
  '2004-04-09',
  '2011-06-20',
  '2015-04-10',
  '2020-12-20',
  '2022-05-22',
  '2020-04-24',
  '2020-07-18',
  '2020-08-28',
  '2020-05-06',
  '2020-07-19',
  '2003-07-27',
  '2004-05-26',
  '2013-06-04',
  '2013-06-26',
  '2013-04-01',
  '2017-07-28',
  '2017-09-08',
  '2017-09-08',
  '2017-09-08',
  '2017-09-08',
  '2019-09-23',
  '2022-05-24',
  '2017-09-09'],
 ['2020-09-15',
  '2020-09-15',
  '2020-03-29',
  '2005-08-18',
  '2005-08-25',
  '2008-07-20',
  '2008-07-11',
  '2008-07-22',
  '2009-09-11',
  '2010-07-18',
  '2004-06-08',
  '2011-08-19',
  '2015-06-09',
  '2021-02-18',
  '2022-07-21',
  '2020-06-23',
  '2020-09-16',
  '2020-10-27',
  '2020-07-05',
  '2020-09-17',
  '2003-09-25',
  '2004-07-25',
  '2013-08-03',
  '2013-08-25',
  '2013-05-31',
  '2017-09-26',
  '2017-11-07',
  '2017-11-07',
  '2017-11-07',
  '2017

------------------------------

In [10]:
def get_bbox(fire):
    '''
    This function returns a list containing the bbox for the geometry of each fire from the geometry column in a geodataframe
    
    args:
    fire : is a geodataframe that contains the geometry (polygon) for each fire
    '''
    #create dataframe with just coordinates of bbox as only column
    fire_bounds = fire.bounds #find bounds for each fire geometry
    fire_bounds['bbox'] = fire_bounds.apply(lambda row: (row['minx'], row['miny'], row['maxx'], row['maxy']), axis=1)
    bbox = fire_bounds['bbox'] 
    
    bbox= tuple(bbox) # ensures bbox is a tuple
    
    return bbox


In [16]:
 %%time 

def get_sat_images(bounds, dates, fires):
    '''
    This function gets image collections for each fire 
    
    args:
    bounds: list of bounding boxes 
    dates: two lists in one- the first is a list of dates that include the start date and end date for the range in query of sat images
    fires: 
    
    This function is designed to be used with the outputs of the previous two functions:
    dates = get_fire_dates(fires) contains two lists- start date is first list, end date is second 
    bounds = get_bbox(fire) 
    '''
    
    #set the start_dates, end_dates, and bboxbounds to be equal to the first two inputs of the function
    start_dates, end_dates = dates #get_fire_dates(FiresWithEuc)
    bboxs = bounds #get_bbox(FiresWithEuc)
    

    api_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
    
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()
    client = Client(cluster)
    client
    
    
    catalog = pystac_client.Client.open(api_url)
    
    
    ###################### transforms bbox from degrees to lat long - depending on your data this may or may not be necessary
    source_crs = 'epsg:3857' # coord system used by fireswitheuc
    target_crs = 'epsg:4326' # Global lat-lon coordinate system
    
    #create the transfromer
    latlon_to_s2_transformer = pyproj.Transformer.from_crs(source_crs, target_crs)
    
    
    projected_coordinates = [] # empty list to store new coordinates
    for point in bboxs:
        projected_minx, projected_miny = latlon_to_s2_transformer.transform(point[1], point[0])
        projected_maxx, projected_maxy = latlon_to_s2_transformer.transform(point[3], point[2])
        projected_coordinates.append((projected_minx, projected_miny, projected_maxx, projected_maxy)) #appends empty list
    
    # to make sure that the new projected coords are tuples
    bbox_new = tuple(projected_coordinates)

    #################################
    
    # interate through the start and end dates, and bboxes
    img_collect = [] #empty list to store image collections
    for i in range(len(fires)):
        search = catalog.search(
            datetime=f"{start_dates[i]}/{end_dates[i]}", 
            collections=["landsat-c2-l2"],
            bbox=bbox_new[i],  
            query=["eo:cloud_cover<5"],
            max_items=1000
        )
        
        #executes search and stores in variable, items
        items = pc.sign(search)
        img_collect.append(items) #stores collections in list, img_collect
        
        len(items)
        
    return img_collect 
        

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 13.8 µs


In [17]:
#defing new variables with results of previous functions
boundingboxes= get_bbox(FiresWithEuc)
fire_dates= get_fire_dates(FiresWithEuc)

#executes get_sat_images for each search
get_sat_images(boundingboxes, fire_dates, FiresWithEuc)

------------------------------------------------------------